# Ridge Regression
Ridge Regression, also known as Tikhonov regularization, is a type of linear regression that includes a regularization term to prevent overfitting. It adds a penalty equivalent to the square of the magnitude of coefficients to the loss function.

## Advantages:
- Prevents Overfitting: The regularization term helps in preventing overfitting by shrinking the coefficients.
- Handles Multicollinearity: Effective in cases where the independent variables are highly collinear.
- Improves Generalization: Can improve the model's generalization performance on unseen data.

## Disadvantages:
- Bias Introduction: Introduces bias into the model, which can reduce interpretability.
- Complexity: Slightly more complex to tune compared to simple linear regression due to the regularization term.

## Use Case:
- Financial Models: Predicting stock prices where features might be highly collinear.
- Medical Research: Predicting disease progression where multiple predictors are related.
- Marketing Analysis: Sales prediction models where multiple marketing channels are interdependent.

## Scaling (necessary)
Yes, scaling is necessary for Ridge Regression because it relies on distance metrics.

## Encoding (necessary)
Yes, categorical features need to be encoded into numerical values.

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import uniform, loguniform

# Read Dataset

In [2]:
df = pd.read_csv('50_StartUp_dataset.csv')
df.head()

,Unnamed: 0,R&D Spend,Administration,Marketing Spend,Profit,Florida,New York
0,0,165349.20,136897.80,471784.10,192261.83,0.0,1.0
1,1,162597.70,151377.59,443898.53,191792.06,0.0,0.0
2,2,153441.51,101145.55,407934.54,191050.39,1.0,0.0
3,3,144372.41,118671.85,383199.62,182901.99,0.0,1.0
4,4,142107.34,91391.77,366168.42,166187.94,1.0,0.0


# get X , Y

In [3]:
x=df.drop('Profit',axis=1)
y=df['Profit']

## Get train, test and valid data

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.1, random_state=42)
x_train, x_valid, y_train, y_valid=train_test_split(x_train,y_train,test_size=.1, random_state=42)

In [5]:
print('x_train shape =',x_train.shape)
print('x_test shape =',x_test.shape)
print('x_valid shape =',x_valid.shape)
print('y_train shape =',y_train.shape)
print('y_test shape =',y_test.shape)
print('y_valid shape =',y_valid.shape)

x_train shape = (40, 6)
x_test shape = (5, 6)
x_valid shape = (5, 6)
y_train shape = (40,)
y_test shape = (5,)
y_valid shape = (5,)


# Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_valid=scaler.transform(x_valid)
x_test=scaler.transform(x_test)

# Train

## Grid Search

In [19]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge = Ridge()

params = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
    'fit_intercept': [True, False],
    'solver': ['auto', 'svd', 'lsqr', 'sag', 'saga']
}

param_grid = {
    'alpha': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
    'fit_intercept': [True, False],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
}

grid_search = GridSearchCV(ridge, param_grid, scoring='r2', cv=5, n_jobs=-1)

# Train the grid search
grid_search.fit(x_train, y_train)  

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'fit_intercept': [True, False],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg', 'sag', 'saga']},
             scoring='r2')

In [21]:
print("Best Hyperparameter Index:", grid_search.best_index_)
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validated Score:", grid_search.best_score_)

Best Hyperparameter Index: 47
Best Hyperparameters: {'alpha': 1, 'fit_intercept': True, 'solver': 'sag'}
Best Cross-Validated Score: 0.9510827092994205


In [9]:
# Get the model with best hyperparameters
model = grid_search.best_estimator_
y_pred = model.predict(x_test)

## Randomized Search

In [27]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV

ridge = Ridge()

params = {
    'alpha': np.logspace(-4, 4, 50),
    'fit_intercept': [True, False],
    'solver': ['auto', 'svd', 'lsqr', 'saga']
}

param_dist = {
    'alpha': np.logspace(-6, 6, 100),
    'fit_intercept': [True, False],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'max_iter': [None, 100, 500, 1000, 5000, 10000]
}

random_search = RandomizedSearchCV(ridge, params, scoring='r2', n_iter=10, cv=5, n_jobs=-1, random_state=42)

# Train the random search
random_search.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=Ridge(),
                   param_distributions={'alpha': array([1.00000000e-04, 1.45634848e-04, 2.12095089e-04, 3.08884360e-04,
       4.49843267e-04, 6.55128557e-04, 9.54095476e-04, 1.38949549e-03,
       2.02358965e-03, 2.94705170e-03, 4.29193426e-03, 6.25055193e-03,
       9.10298178e-03, 1.32571137e-02, 1.93069773e-02, 2.81176870e-02,
       4.09491506e-02, 5.96362332e-02, 8....
       1.67683294e+01, 2.44205309e+01, 3.55648031e+01, 5.17947468e+01,
       7.54312006e+01, 1.09854114e+02, 1.59985872e+02, 2.32995181e+02,
       3.39322177e+02, 4.94171336e+02, 7.19685673e+02, 1.04811313e+03,
       1.52641797e+03, 2.22299648e+03, 3.23745754e+03, 4.71486636e+03,
       6.86648845e+03, 1.00000000e+04]),
                                        'fit_intercept': [True, False],
                                        'solver': ['auto', 'svd', 'lsqr',
                                                   'saga']},
                   random_state=42, scoring='r2')

In [23]:
# print("Best Hyperparameter Index:", random_search.best_index_)
# print("Best Hyperparameters:", random_search.best_params_)
# print("Best Cross-Validated Score:", random_search.best_score_)

Best Hyperparameter Index: 2
Best Hyperparameters: {'solver': 'svd', 'fit_intercept': True, 'alpha': 0.0004498432668969444}
Best Cross-Validated Score: 0.95075525565177


In [12]:
# model = random_search.best_estimator_
# y_pred = model.predict(x_test)

## Train Ridge without search

In [13]:
from sklearn.linear_model import Ridge
model=Ridge(alpha=1, fit_intercept=True, solver='auto')
model.fit(x_train, y_train)

Ridge(alpha=1)

# Check overfiiting

In [14]:
y_train_pred=model.predict(x_train)
r2_score(y_train_pred , y_train)

0.9577500071513344

In [15]:
y_valid_pred=model.predict(x_valid)
r2_score(y_valid_pred , y_valid)

0.9619787079594989

# Evaluate model

In [16]:
y_pred = model.predict(x_test)

## r2_score

In [17]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
r2

0.9075589682962396

## mean_squared_error

In [18]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

62960859.65741825

## mean_absolute_error

In [19]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
mae

5903.649355134783